# Azure AI Search Skill Playground

## Blocks

### Common Setup

In [ ]:
from dotenv import load_dotenv

load_dotenv("../.env")

import sys
sys.path.insert(0, '..\\src')

### Azure AI Search

#### Azure AI Search Index

In [ ]:
from components.search.cognitiveIndex_search import CognitiveSearchIndex
from components.models.config_models import SearchFieldConfig

img_index_name = "sample-images-idx"
select_fields_map = {
    img_index_name: [
        SearchFieldConfig(name="article_id", is_item_id=True),
        SearchFieldConfig(name="category", is_product_category=True),
        SearchFieldConfig(name="prod_name"),
        SearchFieldConfig(name="image_filepath"),
        SearchFieldConfig(name="description", contains_image_description=True),
        SearchFieldConfig(name="gptv_description", contains_image_description=True)
    ]
}
SearchFieldConfig(name="article_id", is_item_id=True)
vector_config_map = {
    img_index_name: ["gptv_hybrid_description_embedding"]
}


### Test Category filter with and without
Using keyword search since it gives the worse results to spot the differences easier

In [ ]:
from components.models.config_models import SearchType

img_search_index = CognitiveSearchIndex(
    img_index_name, 
    select_fields=select_fields_map.get(img_index_name, []),
    vector_config=vector_config_map.get(img_index_name, []),
    search_type=SearchType.keywords
)
print(img_search_index.get_filterable_categories())

In [ ]:
query = "red cozy pants"
print("----WITHOUT category filter----")
print(img_search_index.search(query, results_count=3))
print("----WITH category filter----")
print(img_search_index.search(query, results_count=3, categories=["Trousers"]))

In [ ]:
img_search_index_vector = CognitiveSearchIndex(
    img_index_name, 
    select_fields=select_fields_map.get(img_index_name, []),
    vector_config=vector_config_map.get(img_index_name, []),
    search_type=SearchType.vector
)
print("----WITHOUT category filter----")
print(img_search_index_vector.search(query, results_count=3))
print("----WITH category filter----")
print(img_search_index_vector.search(query, results_count=3, categories=["Trousers"]))

Other examples for validating multiple categories

In [ ]:
img_search_index._get_filter_input(["Jacket","Sweater"])

In [ ]:
query = "something to keep me warm"
print("----WITHOUT category filter----")
print(img_search_index.search(query, results_count=3))
print("----WITH category filter----")
print(img_search_index.search(query, results_count=3, categories=["Jacket","Sweater"]))

### Get all matching items from 1 category

In [ ]:
img_search_index.get_items_by_category("Jacket")

#### Cognitive Search Index Manager

Loads config file and retrieve only indexes defined in config with specific vector and select fields config

In [ ]:
from components.shared.errors import InvalidConfigError
from components.models.config_models import get_valid_config
from components.search.cognitiveIndex_manager import CognitiveSearchIndexManager

valid_config, config = get_valid_config(config_path="../src/components/templates/cognitiveSearch.config.json")

if not valid_config:
    raise InvalidConfigError("Invalid configuration file. Make sure it follows schema format")

index_manager = CognitiveSearchIndexManager(config)

In [ ]:
search_query = "baby outfit with light colors and with stripes"
index_manager.get_product_image_index().search(search_query, 2)

## Post-Filtering Logic

In [ ]:
search_query = "baby outfit with light colors and with stripes"
search_results = index_manager.get_product_image_index().search(search_query, 6)
search_results

In [ ]:
from components.gpt_filtering.post_search_filters import post_search_filter

post_search_filter(search_query, search_results, 
                   items_count=3, 
                   id_key="article_id", 
                   description_keys=["description", "gptv_description"])

### Category summary experiments

In [ ]:
print(img_search_index.get_filterable_categories())

In [ ]:
category = "Dress"
items = img_search_index.get_items_by_category(category)
print(items)

In [ ]:
from components.gpt_filtering.post_search_filters import summarize_collection_items_description

summarize_collection_items_description(category, items)

In [ ]:
from components.utils.blob_storage_utils import get_blob_url

get_blob_url("174057022/0174057022.jpg")